simulacion de dia de trabajo de plataforma, el objetivo es simular un dia de requerimientos en la plataforma 
y poder manipular los requerimientos de entrada de tal manera que la carga de trabajo sea lo mas distribuida
y uniforme posible para los minutos del dia en la plataforma

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
from tqdm import tqdm
import math

In [2]:
l=[ x for x in range(1,101) if x % 10 == 0 ] 
l.insert(0,"dcvfc5zfadfged")
l

['dcvfc5zfadfged', 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [3]:
def rango(df):
    ran = []
    for (index, fila) in df.iterrows():
        cont = 0
        fin = 0 
        for m in fila.values:
            cont+=1
            if(m == 1):
                ini = cont
                cont = 0
                break
        for m in reversed(fila.values):
            cont+=1
            if(m == 1):
                fin = 1441 - cont 
                cont = 0
                break
        if((fin - ini) == 0): # evitamos rango 0 en registros unicos por dia 
            ran.append(1)
            
        else:
            ran.append(fin - ini)
    
    df['rango_min'] = ran
    del ran

In [112]:
# para encotrar primer minuto de aparicion 
def aparicion(df):
    lista_apariciones = []
    for (index, fila) in df.iterrows():
        cont = 0
        for m in fila.values:
            cont+=1
            if(m == 1):
                lista_apariciones.append(cont)
                break
    df['FirstTime'] = lista_apariciones
    del lista_apariciones

In [113]:
def preproceso(df):
    tipo = df["Trigger"].to_numpy()
    df.drop(['HashOwner', 'HashApp', 'Trigger'], axis = 1, inplace = True)
    df.set_index('HashFunction',inplace=True) # para hacer sencillo el reemplazo de # de invocaciones por minuto 
    df=df.where(df <= 1, 1)                 # si se invocó 12veces en un minuto se reeplaza a 1 para analizar periodicidad 
    invocaciones = pd.Series(df.iloc[:, 0:].sum(axis=1)) # llevamos conteo de # de aparicicones en el dia de una funcion
    rango(df)
    df["ivocationDay"] = invocaciones
    df["periodo"] = df.apply(lambda x: 1440 if (x["ivocationDay"]==1) else (x["rango_min"]/(x["ivocationDay"]-1)) , axis = 1) 
    df["periodo"] = df["periodo"].round(2)
    period = df["periodo"].to_numpy()
    df.drop(['periodo', 'rango_min'], axis=1, inplace=True)
    aparicion(df)
    df["period"] = period
    df["Trigger"] = tipo
    df['HashFunction'] = df.index
    df.reset_index(drop = True, inplace = True)
    df = df[['HashFunction','Trigger','period', 'FirstTime']]

    return df # def con HashFuntion como index

In [143]:
# funcion para generar 
def period_list(period, inic):
    l=[1 if x % period == 0 else 0 for x in range(1,1441)] 
    l2 = [0 for x in range(1,abs(period-inic)+1)]
    l = l+l2
    del l[0:abs(period-inic)]
    return l

In [115]:
carga = pd.read_csv("invocations_per_function_md.anon.d05.csv")

In [116]:
dia = preproceso(carga)
dia

,HashFunction,Trigger,period,FirstTime
0,520dbd6bd906840012aa0c4b778743efc7c0ac7b7caf96...,http,5.00,1
1,115ca7a2b5bc290052c3da74cd0347d19c3c67b7d5aa66...,http,5.00,3
2,fa4f840d0cddc391a9afee2b15074f7f92bec15e7ea916...,queue,140.00,101
3,93e6c664773bbec3a7f50a0e92fa7e97401a802dc6eed8...,orchestration,66.95,98
4,740c5c767e4b9978ee59a97d1829cfbaf755a47806a311...,http,70.61,98
...,...,...,...,...
46652,d01feeef6c6a03ba8cc96dbfdcb8898106b18acbe9d100...,http,720.00,679
46653,d43df62e0448cf994831aba36a262443378c03993bbb7b...,http,69.53,170
46654,f1ff5a83374938d5da346b909ae4c6aaf6488f575116c9...,http,1440.00,679
46655,d6a50fa610274244b42a39dcb31f28a699dc289ce5e782...,http,147.75,169


In [179]:
def simulation(df_day):
    record = [0 for x in range(1,1441)]
    overl_minutes = []
    columns = [0 for x in range(1,1441)]
    columns.insert(0,"HashFunction")
    df_result = pd.DataFrame(columns=columns)
    with tqdm(total=len(df_day), file=sys.stdout) as pbar:
        for row in df_day.itertuples(): 
            ini = row.FirstTime
            t_period = period_list(int(row.period), int(ini))
            record = np.add(record,t_period)
            overl_minutes = [i+1 for i, v in enumerate(record) if v > 100]

            if row.FirstTime in overl_minutes:
                while row.FirstTime in overl_minutes:
                    print("entre al ciclo con min "+ str(ini)+"\n", overl_minutes)
                    record = np.subtract(record,t_period)
                    overl_minutes = [i+1 for i, v in enumerate(record) if v > 100]
                    print(overl_minutes)
                    print(record)
                    ini+=1
                    t_period = period_list(int(row.period), int(ini))
                    record = np.add(record,t_period)

            t_period.insert(0,row.HashFunction)
            df_result= df_result.append(pd.DataFrame([t_period], columns=columns),ignore_index=True)
            pbar.set_description('procesando')
            pbar.update(1)
    return df_result
#error,pregunta por un numero de inicio cuando puedo tener mas del limite en min 1 y preguntar por min 2
#solucion, preguntar cuales de los numeros pasan el limite y si el nuevo esta en esa lista aumentar en 1 o 2
#hasta que no este en esa lista 
    

In [180]:
df_mod = simulation(dia)

procesando:   1%|▎                                                               | 251/46657 [00:25<1:18:53,  9.80it/s]entre al ciclo con min 1
 [1]
[]
[100  46  54 ...  35  37  35]
procesando:   1%|▎                                                               | 252/46657 [00:25<1:19:06,  9.78it/s]entre al ciclo con min 1
 [1]
[]
[100  47  54 ...  35  37  35]
procesando:   1%|▎                                                               | 253/46657 [00:25<1:22:47,  9.34it/s]entre al ciclo con min 1
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 2
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 3
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 4
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 5
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 6
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 7
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 8
 [1]
[1]
[101  48  55 ...  36  38  35]
entre

[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 120
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 121
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 122
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 123
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 124
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 125
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 126
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 127
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 128
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 129
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 130
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 131
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 132
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 133
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 134
 [1]
[1]
[101  

 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 250
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 251
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 252
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 253
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 254
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 255
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 256
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 257
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 258
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 259
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 260
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 261
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 262
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 263
 [1]
[1]
[101  48  55 ...  36  38  35]
entre al ciclo con min 264
 [1]
[1]
[

KeyboardInterrupt: 

In [142]:
record = [0 for x in range(1,1441)]
t_period = period_list(66, 98)
#record = np.add(record,t_period)
print(len(record))
print(len(t_period))

print(len(record))

1440
32
1440


In [107]:
first = [1,0,3,0,1]
second = [0,1,0,0,1]
print(np.add(first, second))
print(np.subtract(first, second))

[1 1 3 0 2]
[ 1 -1  3  0  0]


In [74]:
columns = [x for x in range(1,1441)]
columns.insert(0,"HashFunction")

df = pd.DataFrame(columns=columns)
print(df)

Empty DataFrame
Columns: [HashFunction, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
Index: []

[0 rows x 1441 columns]


In [75]:
fila=  [1 for x in range(1,1441)]
fila.insert(0,"154dasdsafgasffsd5ffsdfs4")
print(len(fila))

1441


In [102]:
df = df.append(pd.DataFrame([fila], columns=columns),ignore_index=True)

In [104]:
df

,HashFunction,1,2,3,4,5,6,7,8,9,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440
0,154dasdsafgasffsd5ffsdfs4,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [33]:
lista = period_list(5, 1)
print(lista)

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 

In [36]:
Posx = [i+1 for i, e in enumerate(lista) if e > 1]

In [37]:
print(Posx)

[]


In [5]:
print([1 if x % 5 == 0 else 0 for x in range(1,43)])




[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
